In [1]:
import torch 
import torch.nn as nn  
import numpy as np
from tqdm import tqdm
from torch.cuda.amp import autocast
import cv2
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast
from torch.cuda import amp
from torch.optim import lr_scheduler
import time
import gc
import copy
from collections import defaultdict
from typing import Optional, TypeVar

import matplotlib.pyplot as plt
import albumentations as A
from pathlib import Path
from PIL import Image

from segmentation.config import Configs as CFG
from segmentation.models.unet import unet
from segmentation.scr.utils import losses, metrics, set_seed, transforms, rle_coding
from segmentation.scr.tilling_dataset import random_sub_df

#from segmentation.scr
pd.options.mode.chained_assignment = None

C:\Users\123\AppData\Local\Temp\ipykernel_17340\3381012491.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
train_transform  = transforms.get_transform(transform_type='weak')
val_transform = transforms.get_transform(transform_type='val')

In [98]:
class Tilling_Dataset(Dataset):
    """Creating a dataloader for image tiling"""

    def __init__(
        self,
        name_data: str,
        path_to_df: str,
        use_random_sub: bool = False,
        empty_tile_pct: int = 0,
        sample_limit: Optional[int] = None,
        transform=None,
    ):
        super().__init__()
        self.name_data = name_data
        self.path_to_df = Path(path_to_df)
        self.use_random_sub = use_random_sub
        self.empty_tile_pct = empty_tile_pct
        self.sample_limit = sample_limit
        self.transform = transform

        df = pd.read_csv(self.path_to_df)
        if self.use_random_sub:
            self.df = random_sub_df(
                df=df,
                sample_limit=self.sample_limit,
                empty_tile_pct=self.empty_tile_pct,
            )
        else:
            self.df = df

    def __len__(self) -> int:
        return self.df.shape[0]

    def __getitem__(self, idx) -> tuple:
        img_path, lb_path, _, bbx, _, size = self.df.iloc[idx, :].values
        gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE).astype(np.uint8)
        img = cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB) #(H,W ,C)
        mask = cv2.imread(lb_path, cv2.IMREAD_GRAYSCALE)#.astype('float32')
        if self.transform:
            augmented = self.transform(image=img,mask=mask) # c h w
            img,mask = augmented['image'],augmented['mask']
            
        else:
            img = torch.from_numpy(img)
            img = torch.permute(img, (2 , 0, 1))
            mask = torch.from_numpy(mask) 
        
        img = img.type(torch.float32)
        img = img/255
        mask = mask.type(torch.float32)
        mask = mask / 255

        return img, mask,bbx, size
    
set_seed.set_seed() 
train_loader=  Tilling_Dataset(
    name_data='kidney_1_tilling',
    path_to_df=CFG.path_df_kidney_1_til,
    use_random_sub=True, #use full data
    empty_tile_pct=10,
    sample_limit=100,
    transform=val_transform
)



100
Dataset contains 1528 empty and 12146 non-empty tiles.
Sample 10 empty and 90 non-empty tiles.


In [102]:
train_loader[1][0].shape

torch.Size([3, 512, 512])

In [57]:

train_loader[2][0]


tensor([[[0.1451, 0.1451, 0.1451],
         [0.1529, 0.1529, 0.1529],
         [0.1451, 0.1451, 0.1451],
         ...,
         [0.1608, 0.1608, 0.1608],
         [0.1529, 0.1529, 0.1529],
         [0.1373, 0.1373, 0.1373]],

        [[0.1216, 0.1216, 0.1216],
         [0.1608, 0.1608, 0.1608],
         [0.1569, 0.1569, 0.1569],
         ...,
         [0.1608, 0.1608, 0.1608],
         [0.1412, 0.1412, 0.1412],
         [0.1529, 0.1529, 0.1529]],

        [[0.1529, 0.1529, 0.1529],
         [0.1451, 0.1451, 0.1451],
         [0.1294, 0.1294, 0.1294],
         ...,
         [0.1529, 0.1529, 0.1529],
         [0.1333, 0.1333, 0.1333],
         [0.1412, 0.1412, 0.1412]],

        ...,

        [[0.1765, 0.1765, 0.1765],
         [0.1647, 0.1647, 0.1647],
         [0.1647, 0.1647, 0.1647],
         ...,
         [0.1451, 0.1451, 0.1451],
         [0.1176, 0.1176, 0.1176],
         [0.1137, 0.1137, 0.1137]],

        [[0.1843, 0.1843, 0.1843],
         [0.1529, 0.1529, 0.1529],
         [0.

In [28]:
train_loader.df

,path_img,path_lb,is_empty,bbx,px_stats,size
204,data\kidney_1_tilling\images\0034_0_0_512_512.png,data\kidney_1_tilling\labels\0034_0_0_512_512.png,True,"(0, 0, 512, 512)","[19215, 29577]","(1303, 912)"
287,data\kidney_1_tilling\images\0047_400_791_512_...,data\kidney_1_tilling\labels\0047_400_791_512_...,True,"(400, 791, 512, 512)","[19197, 29966]","(1303, 912)"
654,data\kidney_1_tilling\images\0109_0_0_512_512.png,data\kidney_1_tilling\labels\0109_0_0_512_512.png,False,"(0, 0, 512, 512)","[19321, 34749]","(1303, 912)"
902,data\kidney_1_tilling\images\0150_0_395_512_51...,data\kidney_1_tilling\labels\0150_0_395_512_51...,False,"(0, 395, 512, 512)","[19332, 34254]","(1303, 912)"
1046,data\kidney_1_tilling\images\0174_0_395_512_51...,data\kidney_1_tilling\labels\0174_0_395_512_51...,False,"(0, 395, 512, 512)","[19081, 32674]","(1303, 912)"
...,...,...,...,...,...,...
13108,data\kidney_1_tilling\images\2184_0_791_512_51...,data\kidney_1_tilling\labels\2184_0_791_512_51...,True,"(0, 791, 512, 512)","[20600, 33252]","(1303, 912)"
13136,data\kidney_1_tilling\images\2189_0_395_512_51...,data\kidney_1_tilling\labels\2189_0_395_512_51...,True,"(0, 395, 512, 512)","[20487, 34028]","(1303, 912)"
13150,data\kidney_1_tilling\images\2191_0_791_512_51...,data\kidney_1_tilling\labels\2191_0_791_512_51...,True,"(0, 791, 512, 512)","[20470, 33666]","(1303, 912)"
13230,data\kidney_1_tilling\images\2205_0_0_512_512.png,data\kidney_1_tilling\labels\2205_0_0_512_512.png,True,"(0, 0, 512, 512)","[20515, 33251]","(1303, 912)"


In [49]:
train_loader[8]

(tensor([[[0.0392, 0.0392, 0.0392],
          [0.0667, 0.0667, 0.0667],
          [0.0353, 0.0353, 0.0353],
          ...,
          [0.0667, 0.0667, 0.0667],
          [0.0784, 0.0784, 0.0784],
          [0.0980, 0.0980, 0.0980]],
 
         [[0.0471, 0.0471, 0.0471],
          [0.0471, 0.0471, 0.0471],
          [0.0039, 0.0039, 0.0039],
          ...,
          [0.0863, 0.0863, 0.0863],
          [0.0784, 0.0784, 0.0784],
          [0.0902, 0.0902, 0.0902]],
 
         [[0.0667, 0.0667, 0.0667],
          [0.0510, 0.0510, 0.0510],
          [0.0314, 0.0314, 0.0314],
          ...,
          [0.1059, 0.1059, 0.1059],
          [0.0980, 0.0980, 0.0980],
          [0.0863, 0.0863, 0.0863]],
 
         ...,
 
         [[0.4078, 0.4078, 0.4078],
          [0.4118, 0.4118, 0.4118],
          [0.4196, 0.4196, 0.4196],
          ...,
          [0.0784, 0.0784, 0.0784],
          [0.0824, 0.0824, 0.0824],
          [0.0588, 0.0588, 0.0588]],
 
         [[0.4471, 0.4471, 0.4471],
          [0

In [ ]:
img_path, lb_path, is_empty, bbx, px_stats, size = train_loader[8]



fig, axarr = plt.subplots(ncols=3, figsize=(12, 6))
axarr[0].imshow(torch.permute(img, (1,2,0)).numpy(), cmap="gray")
axarr[1].imshow(color.label2rgb(mask.numpy(), torch.permute(img, (1,2,0)).numpy(), bg_label=0, bg_color=(1.,1.,1.), alpha=0.25))
axarr[2].imshow(mask, vmin=0, interpolation='antialiased', interpolation_stage='rgba')